# Consignes

Intégrer le fichier USvideos.csv qui représente un ensemble de 8000 vidéos Youtube. 

Merger le fichier US_category_id.json pour récupérer le nom des catégories. Il conviendra de bien spécifier l'ID du document.


# Questions 
- 1) Récupérer toutes les vidéos de la chaîne Apple.
- 2) Compter le nombre de catégories différentes
- 3) Si vous ne l'avez pas déjà fait, découper les tags en listes et mettre à jour les tags de chacun des documents avec une requête update.
- 4) Récupérer les vidéos les plus vues.
- 5) Compter le nombre moyen de vues en fonction de la catégorie.
- 6) Récupérer les chaines Youtube avec la plus grande moyenne de likes.

In [1]:
import pandas as pd
import pymongo

In [2]:
client = pymongo.MongoClient('mongo')
database = client['exercices']
collection = database['youtube']

In [3]:
collection.delete_many({})

In [4]:
df_youtube = pd.read_csv("./data/USvideos.csv")
df_youtube.head(2)

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09


## Question 0

### Nettoyer les données

On va commencer par récupérer le nom des catégories des vidéos avec le fichier JSON. Pour cela, on récupère le fichier sous forme de dictionnaires qu'on convertit par la suite en dataframe. Ensuite, on le fusionne avec notre dataframe des vidéos Youtube par la colonne commune category_id.

De plus, on vérifie le type des colonnes, la présence de données duppliquées ou encore de valeurs maquantes.

In [5]:
import json

with open("./data/US_category_id.json") as json_data:
    data_read = json.load(json_data)

df_json = pd.json_normalize(data_read['items'])
df_json = df_json.rename(columns = {'id' : 'category_id', 'snippet.title' : 'category_title'})
df_json['category_id'] = df_json['category_id'].astype('int')

In [6]:
df = pd.merge(df_youtube, df_json, on = 'category_id')
df.drop(['kind', 'etag', 'snippet.channelId', 'snippet.assignable'], axis='columns', inplace=True)
df.head(2)

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date,category_title
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09,Entertainment
1,4MkC65emkG4,Hand In Hand A Benefit For Hurricane Relief | MTV,MTV,24,mtv|video|online|official|tv|television|watch|...,274358,9215,477,838,https://i.ytimg.com/vi/4MkC65emkG4/default.jpg,13.09,Entertainment


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7992 entries, 0 to 7991
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   video_id        7992 non-null   object 
 1   title           7992 non-null   object 
 2   channel_title   7992 non-null   object 
 3   category_id     7992 non-null   int64  
 4   tags            7992 non-null   object 
 5   views           7992 non-null   int64  
 6   likes           7992 non-null   int64  
 7   dislikes        7992 non-null   int64  
 8   comment_total   7992 non-null   int64  
 9   thumbnail_link  7992 non-null   object 
 10  date            7992 non-null   float64
 11  category_title  7992 non-null   object 
dtypes: float64(1), int64(5), object(6)
memory usage: 811.7+ KB


In [8]:
df.duplicated().sum()

0

In [9]:
df.isna().sum().sum()

0

### Importer les données

On doit convertir la dataframe en une liste de dictionnaires. Ainsi, le paramètre 'records' permet de représenter chaque ligne en dictionnaire.

In [10]:
DOCUMENTS = df.to_dict(orient = 'records')

In [ ]:
DOCUMENTS

In [11]:
collection.insert_many(DOCUMENTS)

## Question 1  

#### Récupérer toutes les vidéos de la chaîne Apple.

In [12]:
cur = collection.find({'channel_title' : 'Apple'})

for document in cur :
    print('-----')
    print(document)

-----
{'_id': ObjectId('5fc7fe74d44d83ed40f932f2'), 'video_id': 'K4wEI5zhHB0', 'title': 'iPhone X — Introducing iPhone X — Apple', 'channel_title': 'Apple', 'category_id': 28, 'tags': 'Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lighting|A11 Bionic|augmented reality|emoji|animoji|Face ID|Apple Pay|camera|smartphone', 'views': 7860119, 'likes': 185853, 'dislikes': 26679, 'comment_total': 0, 'thumbnail_link': 'https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg', 'date': 13.09, 'category_title': 'Science & Technology'}
-----
{'_id': ObjectId('5fc7fe74d44d83ed40f93302'), 'video_id': 'K4wEI5zhHB0', 'title': 'iPhone X — Introducing iPhone X — Apple', 'channel_title': 'Apple', 'category_id': 28, 'tags': 'Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lighting|A11 Bionic|augmented reality|emoji|animoji|Face ID|Apple Pay|camera|smartphone', 'views': 12200526, 'likes': 258842, 'dislikes': 44339, 'comment_total': 0, 'thumbnail_link': 'https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg', 'date': 14.09, 'catego

## Question 2

#### Compter le nombre de catégories différentes

Il faut regrouper les vidéos par catégories puis on récupère la taille de la liste. 

In [13]:
cur = collection.aggregate([{"$group" : {"_id" : "$category_id"}}])
len(list(cur))

16

Il y a 16 catégories différentes de vidéos.

## Question 3

#### Si vous ne l'avez pas déjà fait, découper les tags en listes et mettre à jour les tags de chacun des documents avec une requête update.

On a la possibilité de découper les tags avant la création de la collection, lors du nettoyage des données :
```
df_youtube['tags'] = [i.split('|') for i in df_youtube['tags']]
```

L'opération de Map->Reduce permet de récupérer les _id et les tags en parcourant tous les éléments.
Ainsi, pour chaque élément, on peut changer les tags en listes avec la fonction update_one() avec l'_id et le tag correspondant.

J'ai limité l'affichage pour 5 vidéos afin de vérifier le résultat du changement des tags.

In [14]:
map_function = '''function(){emit(this._id, this.tags)}'''
reduce_function = '''function(key,values){return values}'''
out = '''{id, tag}'''
result = collection.map_reduce(
        map_function, 
        reduce_function,
        out)
    

In [15]:
values = []

for i in list(result.find()) :
    for key, value in i.items() :
        values.append(value)
        
    tag = ("".join(values[1])).split('|')
    up = collection.update_one({"_id": values[0]}, {"$set":{"tags": tag}})
    values.clear()

list(collection.find().limit(5))

[{'_id': ObjectId('5fc7fe74d44d83ed40f92cb1'),
  'video_id': 'XpVt6Z1Gjjo',
  'title': '1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED YOUTUBE FOREVER!',
  'channel_title': 'Logan Paul Vlogs',
  'category_id': 24,
  'tags': ['logan paul vlog',
   'logan paul',
   'logan',
   'paul',
   'olympics',
   'logan paul youtube',
   'vlog',
   'daily',
   'comedy',
   'hollywood',
   'parrot',
   'maverick',
   'bird',
   'maverick clothes',
   'diamond play button',
   'logan paul diamond play button',
   '10M subscribers',
   'logan paul 1 year vlogging',
   '1 year vlog',
   'dwarf mamba play button',
   'logan paul history',
   'youtube history',
   '10M',
   '10M plaque',
   'youtube button',
   'diamond button',
   'logang',
   'logang 4 life'],
  'views': 4394029,
  'likes': 320053,
  'dislikes': 5931,
  'comment_total': 46245,
  'thumbnail_link': 'https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg',
  'date': 13.09,
  'category_title': 'Entertainment'},
 {'_id': ObjectId('5fc7fe74d44d83ed40f

## Question 4

#### Récupérer les vidéos les plus vues.

Il faut trier les documents par $views$ par ordre décroissant pour avoir la liste des vidéos les plus vues. 
Ainsi, en limitant le resultat à 5, on obtient les 5 vidéos les plus vues de Youtube.

In [16]:
cur = collection.find().sort([("views", -1)])
list(cur.limit(5))

[{'_id': ObjectId('5fc7fe74d44d83ed40f94173'),
  'video_id': 'MBdVXkSdhwU',
  'title': "BTS (방탄소년단) 'DNA' Official MV",
  'channel_title': 'ibighit',
  'category_id': 10,
  'tags': ['BIGHIT', '빅히트', '방탄소년단', 'BTS', 'BANGTAN', '방탄'],
  'views': 41500672,
  'likes': 2010366,
  'dislikes': 78076,
  'comment_total': 736179,
  'thumbnail_link': 'https://i.ytimg.com/vi/MBdVXkSdhwU/default.jpg',
  'date': 22.09,
  'category_title': 'Music'},
 {'_id': ObjectId('5fc7fe74d44d83ed40f9415f'),
  'video_id': 'MBdVXkSdhwU',
  'title': "BTS (방탄소년단) 'DNA' Official MV",
  'channel_title': 'ibighit',
  'category_id': 10,
  'tags': ['BIGHIT', '빅히트', '방탄소년단', 'BTS', 'BANGTAN', '방탄'],
  'views': 38013692,
  'likes': 1950475,
  'dislikes': 75294,
  'comment_total': 713293,
  'thumbnail_link': 'https://i.ytimg.com/vi/MBdVXkSdhwU/default.jpg',
  'date': 21.09,
  'category_title': 'Music'},
 {'_id': ObjectId('5fc7fe74d44d83ed40f9404a'),
  'video_id': 'tt2k8PGm-TI',
  'title': 'ZAYN - Dusk Till Dawn ft. Sia',
  

## Question 5

#### Compter le nombre moyen de vues en fonction de la catégorie.

On regroupe les vidéos par catégories puis avec l'opérateur $avg$ qu'on applique sur le paramètre $views$, on obtient la moyenne de vues par catégories. 

In [20]:
cur = collection.aggregate([{"$group" : {"_id" : "$category_title", "Moyenne de vues" : {"$avg" : '$views'}}}])
list(cur)

[{'_id': 'Entertainment', 'Moyenne de vues': 1154868.4397251718},
 {'_id': 'Education', 'Moyenne de vues': 547582.4700598803},
 {'_id': 'People & Blogs', 'Moyenne de vues': 971532.7573696146},
 {'_id': 'Film & Animation', 'Moyenne de vues': 1039472.6560846561},
 {'_id': 'Music', 'Moyenne de vues': 1176553.612},
 {'_id': 'Comedy', 'Moyenne de vues': 1240073.294039735},
 {'_id': 'News & Politics', 'Moyenne de vues': 540955.5569823436},
 {'_id': 'Nonprofits & Activism', 'Moyenne de vues': 1110334.2142857143},
 {'_id': 'Sports', 'Moyenne de vues': 728434.3243902439},
 {'_id': 'Autos & Vehicles', 'Moyenne de vues': 607693.8706896552},
 {'_id': 'Gaming', 'Moyenne de vues': 681081.8170731707},
 {'_id': 'Shows', 'Moyenne de vues': 8492.5},
 {'_id': 'Pets & Animals', 'Moyenne de vues': 651404.3879310344},
 {'_id': 'Howto & Style', 'Moyenne de vues': 537665.1806674339},
 {'_id': 'Science & Technology', 'Moyenne de vues': 924730.548828125},
 {'_id': 'Travel & Events', 'Moyenne de vues': 464041.08

## Question 6 

#### Récupérer les chaines Youtube avec la plus grande moyenne de likes.

On regroupe les vidéos par chaine puis avec l'opérateur $avg$ qu'on applique sur le paramètre $likes$, on obtient la moyenne de likes par chaine Youtube. On trie ensuite par moyenne de likes en ordre décroissant afin d'avoir en premiers les chaines avec la plus grande moyenne de likes.

In [22]:
cur = collection.aggregate([{"$group" : {"_id" : "$channel_title", "moyenne_likes" : {"$avg" : '$likes'}}}, {"$sort" : {"moyenne_likes" : -1}}])
list(cur)

[{'_id': 'ZaynVEVO', 'moyenne_likes': 1431683.0},
 {'_id': 'ibighit', 'moyenne_likes': 1371766.25},
 {'_id': 'melanie martinez', 'moyenne_likes': 911871.5},
 {'_id': 'BETNetworks', 'moyenne_likes': 769687.625},
 {'_id': 'jypentertainment', 'moyenne_likes': 758826.6666666666},
 {'_id': 'Logan Paul Vlogs', 'moyenne_likes': 653443.5384615385},
 {'_id': 'Wengie', 'moyenne_likes': 567235.2},
 {'_id': 'Alesso', 'moyenne_likes': 509417.28571428574},
 {'_id': 'PewDiePie', 'moyenne_likes': 508647.8},
 {'_id': 'shakiraVEVO', 'moyenne_likes': 434302.14285714284},
 {'_id': 'Ed Sheeran', 'moyenne_likes': 395654.1666666667},
 {'_id': 'Dude Perfect', 'moyenne_likes': 389697.1111111111},
 {'_id': 'Conor Maynard', 'moyenne_likes': 389273.0},
 {'_id': 'TaylorSwiftVEVO', 'moyenne_likes': 367455.6666666667},
 {'_id': 'Liza Koshy', 'moyenne_likes': 365146.9583333333},
 {'_id': 'Linkin Park', 'moyenne_likes': 356475.7272727273},
 {'_id': 'jbalvinVEVO', 'moyenne_likes': 345130.4285714286},
 {'_id': 'Primitiv

La chaine Youtube ZaynVEVO a la plus grande moyenne de likes.